# Assignment 3: Image Colourization [50 Pt]

In this assignment, you will build models to perform image colourization. That is, given a greyscale image, we wish to predict the colour at each pixel. Image colourization is a difficult problem for many reasons, one of which being that it is ill-posed: for a single greyscale image, there can be multiple, equally valid colourings.

To keep the training time manageable we will use the CIFAR-10 data set, which consists of images of size 32x32 pixels. For most of the questions we will use a subset of the dataset. The data loading script is included with the notebooks, and should download automatically the first time it is loaded.

We will be starting with a convolutional autoencoder and tweaking it along the way to improve our perforamnce. Then as a second part of the assignment we will compare the autoencoder approach to conditional generative adversarial networks (cGANs).

In the process, you are expected to learn to:

1. Clean and process the dataset and create greyscale images.
2. Implement and modify an autoencoder architecture.
3. Tune the hyperparameters of an autoencoder.
4. Implement skip connections and other techniques to improve performance.
5. Implement a cGAN and compare with an autoencoder.
6. Improve on the cGAN by trying one of several techniques to enhance training.

*This assignment is based on an assignment developed by Prof. Lisa Zhang and Prof. Jimmy Ba.*

### What to submit

Submit an HTML file containing all your code, outputs, and write-up
from parts A and B. Please take extra effort to make your answers and submissions readable. Do not display unnecessary outputs, only the ones that are important for supporting your answers.

You can produce a HTML file directly from Google Colab. The Colab instructions are provided at the end of this document.

**Do not submit any other files produced by your code.**

Include a link to your colab file in your submission.

Please use Google Colab to complete this assignment. If you want to use Jupyter Notebook, please complete the assignment and upload your Jupyter Notebook file to Google Colab for submission.

## Colab Link

Include a link to your Colab file here. If you would like the TA to look at your
Colab file in case your solutions are cut off, **please make sure that your Colab
file is publicly accessible at the time of submission**.

In [ ]:
# # TO BE COMPLETED

# http://

# PART A - Autoencoder
In this part we will construct and compare different autoencoder models for the image colourization task.

#### Helper code

Provided are some helper functions for loading and preparing the data. Note that you will need to use the Colab GPU for this assignment.

In [ ]:
"""
Colourization of CIFAR-10 Horses via classification.
"""
import argparse
import math
import time

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import numpy.random as npr
import scipy.misc
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable


In [ ]:
######################################################################
# Setup working directory
######################################################################
%mkdir -p /content/a3/
%cd /content/a3


In [ ]:
######################################################################
# Helper functions for loading data
######################################################################
# adapted from
# https://github.com/fchollet/keras/blob/master/keras/datasets/cifar10.py

import os
import pickle
import sys
import tarfile

import numpy as np
from PIL import Image
from six.moves.urllib.request import urlretrieve


def get_file(fname, origin, untar=False, extract=False, archive_format="auto", cache_dir="data"):
    datadir = os.path.join(cache_dir)
    if not os.path.exists(datadir):
        os.makedirs(datadir)

    if untar:
        untar_fpath = os.path.join(datadir, fname)
        fpath = untar_fpath + ".tar.gz"
    else:
        fpath = os.path.join(datadir, fname)

    print("File path: %s" % fpath)
    if not os.path.exists(fpath):
        print("Downloading data from", origin)

        error_msg = "URL fetch failure on {}: {} -- {}"
        try:
            try:
                urlretrieve(origin, fpath)
            except URLError as e:
                raise Exception(error_msg.format(origin, e.errno, e.reason))
            except HTTPError as e:
                raise Exception(error_msg.format(origin, e.code, e.msg))
        except (Exception, KeyboardInterrupt) as e:
            if os.path.exists(fpath):
                os.remove(fpath)
            raise

    if untar:
        if not os.path.exists(untar_fpath):
            print("Extracting file.")
            with tarfile.open(fpath) as archive:
                archive.extractall(datadir)
        return untar_fpath

    if extract:
        _extract_archive(fpath, datadir, archive_format)

    return fpath


def load_batch(fpath, label_key="labels"):
    """Internal utility for parsing CIFAR data.
    # Arguments
        fpath: path the file to parse.
        label_key: key for label data in the retrieve
            dictionary.
    # Returns
        A tuple `(data, labels)`.
    """
    f = open(fpath, "rb")
    if sys.version_info < (3,):
        d = pickle.load(f)
    else:
        d = pickle.load(f, encoding="bytes")
        # decode utf8
        d_decoded = {}
        for k, v in d.items():
            d_decoded[k.decode("utf8")] = v
        d = d_decoded
    f.close()
    data = d["data"]
    labels = d[label_key]

    data = data.reshape(data.shape[0], 3, 32, 32)
    return data, labels


def load_cifar10(transpose=False):
    """Loads CIFAR10 dataset.
    # Returns
        Tuple of Numpy arrays: `(x_train, y_train), (x_test, y_test)`.
    """
    dirname = "cifar-10-batches-py"
    origin = "http://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz"
    path = get_file(dirname, origin=origin, untar=True)

    num_train_samples = 50000

    x_train = np.zeros((num_train_samples, 3, 32, 32), dtype="uint8")
    y_train = np.zeros((num_train_samples,), dtype="uint8")

    for i in range(1, 6):
        fpath = os.path.join(path, "data_batch_" + str(i))
        data, labels = load_batch(fpath)
        x_train[(i - 1) * 10000 : i * 10000, :, :, :] = data
        y_train[(i - 1) * 10000 : i * 10000] = labels

    fpath = os.path.join(path, "test_batch")
    x_test, y_test = load_batch(fpath)

    y_train = np.reshape(y_train, (len(y_train), 1))
    y_test = np.reshape(y_test, (len(y_test), 1))

    if transpose:
        x_train = x_train.transpose(0, 2, 3, 1)
        x_test = x_test.transpose(0, 2, 3, 1)
    return (x_train, y_train), (x_test, y_test)

In [ ]:
# Download CIFAR dataset
m = load_cifar10()

## Part 1 Data Preparation [4 pt]

To start off run the above code to load the CIFAR dataset and then work through the following questions/tasks.

### Part (i) [2pt EXPLORATORY] - load dataset
Verify that the dataset has loaded correctly. How many samples do we have? How is the data organized?


The CIFAR-10 dataset, which is a collection of 60,000 32x32 color images in 10 different classes. The load_cifar10 function returns a tuple containing two tuples:

> The first tuple contains the training data:
> * `x_train`: a NumPy array of shape (50000, 3, 32, 32) containing the training images. Each image is represented as a 3x32x32 array, where the 3 corresponds to the RGB channels, and the 32x32 corresponds to the height and width of the image.
> * `y_train`: a NumPy array of shape (50000, 1) containing the labels for the training images. Each label is an integer from 0 to 9, corresponding to one of the 10 classes

> The second tuple contains the test data:
> * `x_test`: a NumPy array of shape (10000, 3, 32, 32) containing the test images.
> * `y_test`: a NumPy array of shape (10000, 1) containing the labels for the test images.

In [ ]:
type(m)

In [ ]:
type(m[0])

In [ ]:
### X_train

In [ ]:
m[0][0].shape

In [ ]:
### y_train

In [ ]:
m[0][1].shape

In [ ]:
### X_test

In [ ]:
m[1][0].shape

In [ ]:
### y_test

In [ ]:
m[1][1].shape

### Part (ii) [2pt EXPLORATORY] - select only horses

Provided below is sample code to preproces the data to select only images of horses, which will help to simplify the goals of the assignment. The function also converts the colour images to greyscale to create our input data.

In [ ]:
# select a single category.
HORSE_CATEGORY = 7

# convert colour images into greyscale
def process(xs, ys, max_pixel=256.0, downsize_input=False):
    """
    Pre-process CIFAR10 images by taking only the horse category,
    shuffling, and have colour values be bound between 0 and 1

    Args:
      xs: the colour RGB pixel values
      ys: the category labels
      max_pixel: maximum pixel value in the original data
    Returns:
      xs: value normalized and shuffled colour images
      grey: greyscale images, also normalized so values are between 0 and 1
    """
    xs = xs / max_pixel
    xs = xs[np.where(ys == HORSE_CATEGORY)[0], :, :, :]
    npr.shuffle(xs)

    grey = np.mean(xs, axis=1, keepdims=True)

    if downsize_input:
        downsize_module = nn.Sequential(
            nn.AvgPool2d(2),
            nn.AvgPool2d(2),
            nn.Upsample(scale_factor=2),
            nn.Upsample(scale_factor=2),
        )
        xs_downsized = downsize_module.forward(torch.from_numpy(xs).float())
        xs_downsized = xs_downsized.data.numpy()
        return (xs, xs_downsized)
    else:
        return (xs, grey)


The provided get_batch function creates a dataloader (or function) to batch the samples.

In [ ]:
# dataloader for batching samples

def get_batch(x, y, batch_size):
    """
    Generated that yields batches of data

    Args:
      x: input values
      y: output values
      batch_size: size of each batch
    Yields:
      batch_x: a batch of inputs of size at most batch_size
      batch_y: a batch of outputs of size at most batch_size
    """
    N = np.shape(x)[0]
    assert N == np.shape(y)[0]
    for i in range(0, N, batch_size):
        batch_x = x[i : i + batch_size, :, :, :]
        batch_y = y[i : i + batch_size, :, :, :]
        yield (batch_x, batch_y)


Run the above helper code and call the appropriate fucntion to verify and visualize that we are able to generate different batches of data with the correct class of images (i.e., horses).

Write code to visualize 5 train and test/val images.

In [ ]:
# code to load different batches of horse dataset

print("Loading data...")
(x_train, y_train), (x_test, y_test) = load_cifar10()

print("Transforming data...")
train_rgb, train_grey = process(x_train, y_train)
test_rgb, test_grey = process(x_test, y_test)


In [ ]:
# shape of data and labels before selection
print(x_train.shape, y_train.shape)

In [ ]:
# shape of training data
print('Training Data: ', train_rgb.shape, train_grey.shape)
# shape of testing data
print('Testing Data: ', test_rgb.shape, test_grey.shape)

Load Batches

In [ ]:
# TO BE COMPLETED

import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torch.nn import Sequential, AvgPool2d, Upsample

# (Include the helper functions and load_cifar10 function here)

# Load CIFAR dataset
(x_train, y_train), (x_test, y_test) = load_cifar10()

# Preprocess training images
(xs_train, grey_train) = process(x_train, y_train)

# Preprocess test images
(xs_test, grey_test) = process(x_test, y_test)
# Define a function to plot images
def plot_images(images, titles, rows, cols):
    fig, axes = plt.subplots(rows, cols, figsize=(10, 5))
    for i, ax in enumerate(axes.flatten()):
        if i < len(images):
            img = images[i].transpose(1, 2, 0)  # Transpose the image
            ax.imshow(img)
            ax.set_title(titles[i])
        ax.axis('off')
    plt.tight_layout()
    plt.show()

# Plot 5 training images
plot_images(xs_train[:5], ['Horse'] * 5, 1, 5)


Write code to visualize 5 train/test images

In [ ]:

# Plot 5 test images
plot_images(xs_test[:5], ['Horse'] * 5, 1, 5)


## Part 2 Image Colourization as Regression [4 pt]

There are many ways to frame the problem of image colourization as a machine learning problem. One naive approach is to frame it as a regression problem, where we build a model to predict the RGB intensities at each pixel given the greyscale input. In this case, the outputs are continuous, and so squared error can be used to train the model.

In this section, you will be the training neural networks using cloud GPUs. Run the helper code and answer the questions that follow.

#### Helper Code

Regression Architecture

In [ ]:
class RegressionCNN(nn.Module):
    def __init__(self, kernel, num_filters):
        # first call parent's initialization function
        super().__init__()
        padding = kernel // 2

        self.downconv1 = nn.Sequential(
            nn.Conv2d(1, num_filters, kernel_size=kernel, padding=padding),
            nn.BatchNorm2d(num_filters),
            nn.ReLU(),
            nn.MaxPool2d(2),)
        self.downconv2 = nn.Sequential(
            nn.Conv2d(num_filters, num_filters*2, kernel_size=kernel, padding=padding),
            nn.BatchNorm2d(num_filters*2),
            nn.ReLU(),
            nn.MaxPool2d(2),)

        self.rfconv = nn.Sequential(
            nn.Conv2d(num_filters*2, num_filters*2, kernel_size=kernel, padding=padding),
            nn.BatchNorm2d(num_filters*2),
            nn.ReLU())

        self.upconv1 = nn.Sequential(
            nn.Conv2d(num_filters*2, num_filters, kernel_size=kernel, padding=padding),
            nn.BatchNorm2d(num_filters),
            nn.ReLU(),
            nn.Upsample(scale_factor=2),)
        self.upconv2 = nn.Sequential(
            nn.Conv2d(num_filters, 3, kernel_size=kernel, padding=padding),
            nn.BatchNorm2d(3),
            nn.ReLU(),
            nn.Upsample(scale_factor=2),)
        self.finalconv = nn.Conv2d(3, 3, kernel_size=kernel, padding=padding)

    def forward(self, x):
        out = self.downconv1(x)
        out = self.downconv2(out)
        out = self.rfconv(out)
        out = self.upconv1(out)
        out = self.upconv2(out)
        out = self.finalconv(out)
        return out

In [ ]:
import torch
if torch.backends.mps.is_available():
    mps_device = torch.device("mps")
    x = torch.ones(1, device=mps_device)
    print (x)
else:
    print ("MPS device not found.")

Training code

In [ ]:
class AttrDict(dict):
    def __init__(self, *args, **kwargs):
        super(AttrDict, self).__init__(*args, **kwargs)
        self.__dict__ = self

def get_torch_vars(xs, ys, gpu=False, mps_device=None):
    """
    Helper function to convert numpy arrays to pytorch tensors.
    If GPU is used, move the tensors to GPU.

    Args:
      xs (float numpy tensor): greyscale input
      ys (int numpy tensor): rgb as labels
      gpu (bool): whether to move pytorch tensor to GPU
      mps_device (torch.device): device for MPS backend
    Returns:
      Variable(xs), Variable(ys)
    """
    xs = torch.from_numpy(xs).float()
    ys = torch.from_numpy(ys).float()
    if gpu and mps_device is not None:
        xs = xs.to(mps_device)
        ys = ys.to(mps_device)
    return Variable(xs), Variable(ys)


def train(args, gen=None):

    # Numpy random seed
    npr.seed(args.seed)

    # Save directory
    save_dir = "outputs/" + args.experiment_name

    # LOAD THE MODEL
    if gen is None:
        Net = globals()[args.model]
        gen = Net(args.kernel, args.num_filters)

    # LOSS FUNCTION
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(gen.parameters(), lr=args.learn_rate)

    # DATA
    print("Loading data...")
    (x_train, y_train), (x_test, y_test) = load_cifar10()

    print("Transforming data...")
    train_rgb, train_grey = process(x_train, y_train, downsize_input=args.downsize_input)
    test_rgb, test_grey = process(x_test, y_test, downsize_input=args.downsize_input)

    # Create the outputs folder if not created already
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    print("Beginning training ...")
    if args.gpu:
        if torch.backends.mps.is_available():
            mps_device = torch.device("mps")
            gen.to(mps_device)
        else:
            print("MPS device not available. Falling back to CPU.")
    
    start = time.time()

    train_losses = []
    valid_losses = []
    valid_accs = []
    for epoch in range(args.epochs):
        # Train the Model
        gen.train()  # Change model to 'train' mode
        losses = []
        for i, (xs, ys) in enumerate(get_batch(train_grey, train_rgb, args.batch_size)):
            images, labels = get_torch_vars(xs, ys, args.gpu, mps_device if args.gpu else None)
            # Forward + Backward + Optimize
            optimizer.zero_grad()
            outputs = gen(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            losses.append(loss.data.item())

        train_losses.append(np.mean(losses))

        # Validate the Model
        gen.eval()  # Change model to 'eval' mode
        val_losses = []
        for i, (xs, ys) in enumerate(get_batch(test_grey, test_rgb, args.batch_size)):
            images, labels = get_torch_vars(xs, ys, args.gpu, mps_device if args.gpu else None)
            outputs = gen(images)
            loss = criterion(outputs, labels)
            val_losses.append(loss.data.item())

        valid_losses.append(np.mean(val_losses))
        
        print(f'Epoch [{epoch+1}/{args.epochs}], '
              f'Training Loss: {train_losses[-1]:.4f}, '
              f'Validation Loss: {valid_losses[-1]:.4f}')

        if args.plot:
            visual(images, labels, outputs, args.gpu, 1)

    # Plot the training and validation losses
    plt.figure(figsize=(10, 5))
    plt.plot(train_losses, label='Training Loss')
    plt.plot(valid_losses, label='Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Training and Validation Losses')
    plt.legend()
    plt.show()

    return gen


Training visualization code

In [ ]:
# visualize 5 train/test images
def visual(img_grey, img_real, img_fake, gpu = 0, flag_torch = 0):

  if gpu:
    img_grey = img_grey.cpu().detach()
    img_real = img_real.cpu().detach()
    img_fake = img_fake.cpu().detach()

  if flag_torch:
    img_grey = img_grey.numpy()
    img_real = img_real.numpy()
    img_fake = img_fake.numpy()

  if flag_torch == 2:
    img_real = np.transpose(img_real[:, :, :, :, :], [0, 4, 2, 3, 1]).squeeze()
    img_fake = np.transpose(img_fake[:, :, :, :, :], [0, 4, 2, 3, 1]).squeeze()

  #correct image structure
  img_grey = np.transpose(img_grey[:5, :, :, :], [0, 2, 3, 1]).squeeze()
  img_real = np.transpose(img_real[:5, :, :, :], [0, 2, 3, 1])
  img_fake = np.transpose(img_fake[:5, :, :, :], [0, 2, 3, 1])

  for i in range(5):
      ax = plt.subplot(3, 5, i + 1)
      ax.imshow(img_grey[i], cmap='gray')
      ax.axis("off")
      ax = plt.subplot(3, 5, i + 1 + 5)
      ax.imshow(img_real[i])
      ax.axis("off")
      ax = plt.subplot(3, 5, i + 1 + 10)
      ax.imshow(img_fake[i])
      ax.axis("off")
  plt.show()

Main training loop for regression CNN

In [ ]:
args

### Part (i) [2pt EXPLORATORY] - Describe model
Describe the model RegressionCNN. How many convolution layers does it have? What are the filter sizes and number of filters at each layer?

> The network has 6 convolutional layers.

```python
{'gpu': True,
 'valid': False,
 'checkpoint': '',
 'colours': './data/colours/colour_kmeans24_cat7.npy',
 'model': 'RegressionCNN',
 'kernel': 3,
 'num_filters': 32,
 'learn_rate': 0.001,
 'batch_size': 100,
 'epochs': 25,
 'seed': 0,
 'plot': True,
 'experiment_name': 'colourization_cnn',
 'visualize': False,
 'downsize_input': False}
```
> With the provided arguments (`args`), the specified kernel = 3 and num_filters = 32. Given this, the filter sizes and number of filters at each convolutional layer in the RegressionCNN model would be as follows:

```python
     self.downconv1 = nn.Sequential(
            nn.Conv2d(1, num_filters, kernel_size=kernel, padding=padding),
            nn.BatchNorm2d(num_filters),
            nn.ReLU(),
            nn.MaxPool2d(2),)
        self.downconv2 = nn.Sequential(
            nn.Conv2d(num_filters, num_filters*2, kernel_size=kernel, padding=padding),
            nn.BatchNorm2d(num_filters*2),
            nn.ReLU(),
            nn.MaxPool2d(2),)

        self.rfconv = nn.Sequential(
            nn.Conv2d(num_filters*2, num_filters*2, kernel_size=kernel, padding=padding),
            nn.BatchNorm2d(num_filters*2),
            nn.ReLU())

        self.upconv1 = nn.Sequential(
            nn.Conv2d(num_filters*2, num_filters, kernel_size=kernel, padding=padding),
            nn.BatchNorm2d(num_filters),
            nn.ReLU(),
            nn.Upsample(scale_factor=2),)
        self.upconv2 = nn.Sequential(
            nn.Conv2d(num_filters, 3, kernel_size=kernel, padding=padding),
            nn.BatchNorm2d(3),
            nn.ReLU(),
            nn.Upsample(scale_factor=2),)
        self.finalconv = nn.Conv2d(3, 3, kernel_size=kernel, padding=padding)
```



> * `downconv1`: kernel size = 3, 1 input channel, 32 output channels.
> * `downconv2`: kernel size = 3, 32 input channels, 64 output channels.
> * `rfconv`: kernel size = 3, 64 input channels, 64 output channels.
> * `upconv1`: kernel size = 3, 64 input channels, 32 output channels.
> * `upconv2`: kernel size = 3, 32 input channels, 3 output channels.
> * `finalconv`: kernel size = 3, 3 input channels, 3 output channels.

### Part (ii) [2pt EXPLORATORY] - Run Regression Training Code
Run the regression training code (should run without errors). This will generate some images. How many epochs are we training the CNN model in the given setting?

> 25 epochs

In [ ]:
#Main training loop for CNN
args = AttrDict()
args_dict = {
    "gpu": True,
    "valid": False,
    "checkpoint": "",
    "colours": "./data/colours/colour_kmeans24_cat7.npy",
    "model": "RegressionCNN",
    "kernel": 3,
    "num_filters": 32,
    'learn_rate':0.001,
    "batch_size": 100,
    "epochs": 25,
    "seed": 0,
    "plot": True,
    "experiment_name": "colourization_cnn",
    "visualize": False,
    "downsize_input": False,
}

args.update(args_dict)
cnn = train(args)

### Part (iii) [1pt RESULTS] - Retrain with more/less epochs
Re-train a couple of new models using a different number of training epochs. You may train each new models in a new code cell by copying and modifying the code from the last notebook cell. Comment on how the results (output images, training loss) change as we increase or decrease the number of epochs.
> Note that for every epoch we have three rows of images:
> * The first row displays the grayscale input images (`img_grey`).
> * The second row displays the real, ground truth color images (`img_real`).
> * The third row of images corresponds to the color images generated by the model based on the grayscale input images (`img_fake`). These generated images can be compared to the ground truth color images in the second row to see how well the model is performing.

> As we increase the number of epochs, the model has more opportunities to learn from the trainin data, which will likely result in a decrease in training loss. However, after a certain point, the model may start to overfit to the training data.
> On the other hand, if you decrease the number of epochs, the model will have fewer opportunities to learn from the training data, which may result in a higher training loss. However, it also means that the model is less likely to overfit to the training data.
> As for the output images, as you increase the number of epochs, we see that the output images to become more accurate representations of the ground truth images, since the model has more opportunities to learn from the training data. Conversely, as you decrease the number of epochs, the output images to are less accurate representations of the ground truth images (more pixelated).

In [ ]:
#Main training loop for CNN
args = AttrDict()
args_dict = {
    "gpu": True,
    "valid": False,
    "checkpoint": "",
    "colours": "./data/colours/colour_kmeans24_cat7.npy",
    "model": "RegressionCNN",
    "kernel": 3,
    "num_filters": 32,
    'learn_rate':0.001,
    "batch_size": 100,
    "epochs": 10,
    "seed": 0,
    "plot": True,
    "experiment_name": "colourization_cnn",
    "visualize": False,
    "downsize_input": False,
}

args.update(args_dict)
cnn = train(args)

In [ ]:
#Main training loop for CNN
args = AttrDict()
args_dict = {
    "gpu": True,
    "valid": False,
    "checkpoint": "",
    "colours": "./data/colours/colour_kmeans24_cat7.npy",
    "model": "RegressionCNN",
    "kernel": 3,
    "num_filters": 32,
    'learn_rate':0.001,
    "batch_size": 100,
    "epochs": 50,
    "seed": 0,
    "plot": True,
    "experiment_name": "colourization_cnn",
    "visualize": False,
    "downsize_input": False,
}

args.update(args_dict)
cnn = train(args)

### Part (iv) [1pt MODEL]

Modify the convolutional neural network to include additional convolutional layers. Specifically, add an additional convolutional layer with the same number of filters as the previous layer and the same kernel size. Ensure that the appropriate parameters are modified (e.g. input and output channels, padding) to accommodate this change.


> In the case of the additional layers I've added, there are no changes required in the input/output channels or padding because these additional layers are internal to the respective blocks (downconv1, downconv2, and upconv1).

> * For `downconv1`, the additional layer takes as input the number of filters from the previous layer (`num_filters`), and outputs the same number of filters (`num_filters`).
> * For `downconv2`, the additional layer takes as input the number of filters from the previous layer (`num_filters*2`), and outputs the same number of filters (`num_filters*2`).
> * For `upconv1`, the additional layer takes as input the number of filters from the previous layer (`num_filters`), and outputs the same number of filters (`num_filters`).

> In each case, since the input and output channels are the same for the additional layer, there is no need to adjust the padding. The padding remains the same as it was for the existing layers in the block, which is `kernel // 2`.

#### Note that the kernel needs to be restarted here so that there is no namespace clashing with `RegressionCNN`

In [ ]:
# TO BE COMPLETED


class RegressionCNN_modified(nn.Module):
    def __init__(self, kernel, num_filters):
        # first call parent's initialization function
        super().__init__()
        padding = kernel // 2

        self.downconv1 = nn.Sequential(
            nn.Conv2d(1, num_filters, kernel_size=kernel, padding=padding),
            nn.BatchNorm2d(num_filters),
            nn.ReLU(),
            nn.Conv2d(num_filters, num_filters, kernel_size=kernel, padding=padding),  # New layer
            nn.BatchNorm2d(num_filters),
            nn.ReLU(),
            nn.MaxPool2d(2),)
        self.downconv2 = nn.Sequential(
            nn.Conv2d(num_filters, num_filters*2, kernel_size=kernel, padding=padding),
            nn.BatchNorm2d(num_filters*2),
            nn.ReLU(),
            nn.Conv2d(num_filters*2, num_filters*2, kernel_size=kernel, padding=padding),  # New layer
            nn.BatchNorm2d(num_filters*2),
            nn.ReLU(),
            nn.MaxPool2d(2),)

        self.rfconv = nn.Sequential(
            nn.Conv2d(num_filters*2, num_filters*2, kernel_size=kernel, padding=padding),
            nn.BatchNorm2d(num_filters*2),
            nn.ReLU())

        self.upconv1 = nn.Sequential(
            nn.Conv2d(num_filters*2, num_filters, kernel_size=kernel, padding=padding),
            nn.BatchNorm2d(num_filters),
            nn.ReLU(),
            nn.Conv2d(num_filters, num_filters, kernel_size=kernel, padding=padding),  # New layer
            nn.BatchNorm2d(num_filters),
            nn.ReLU(),
            nn.Upsample(scale_factor=2),)
        self.upconv2 = nn.Sequential(
            nn.Conv2d(num_filters, 3, kernel_size=kernel, padding=padding),
            nn.BatchNorm2d(3),
            nn.ReLU(),
            nn.Upsample(scale_factor=2),)
        self.finalconv = nn.Conv2d(3, 3, kernel_size=kernel, padding=padding)

    def forward(self, x):
        out = self.downconv1(x)
        out = self.downconv2(out)
        out = self.rfconv(out)
        out = self.upconv1(out)
        out = self.upconv2(out)
        out = self.finalconv(out)
        return out


args = AttrDict()
args_dict = {
    "gpu": True,
    "valid": False,
    "checkpoint": "",
    "colours": "./data/colours/colour_kmeans24_cat7.npy",
    "model": "RegressionCNN_modified",
    "kernel": 3,
    "num_filters": 32,
    'learn_rate':0.001,
    "batch_size": 100,
    "epochs": 25,
    "seed": 0,
    "plot": True,
    "experiment_name": "colourization_cnn",
    "visualize": False,
    "downsize_input": False,
}

args.update(args_dict)
cnn = train(args)

## Part 3 Skip Connections [8 pt]
A skip connection in a neural network is a connection which skips one or more layer and connects to a later layer. In this section we will be incorporating skip connections to improve on our model architecture.

### Part (i) [3pt MODEL] - Redefine Network
Add a skip connection from the first layer to the last, second layer to the second last, etc.
That is, the final convolution should have both the output of the previous layer and the initial greyscale input as input. This type of skip-connection architecture and is called a "UNet". Following the CNN class that you have completed, complete the __init__ and forward methods of the UNet class.
Hint: You will need to use the function torch.cat to add skip connections.

In [ ]:
# #complete the code

# class UNet(nn.Module):
#     def __init__(self, kernel, num_filters, num_colours=3, num_in_channels=1):
#         super().__init__()

#         # Useful parameters
#         stride = 2
#         padding = kernel // 2
#         output_padding = 1

#         ############### YOUR CODE GOES HERE ###############
#         ###################################################








#     def forward(self, x):
#         ############### YOUR CODE GOES HERE ###############
#         ###################################################










#         return out

In [ ]:
import torch.nn as nn
import torch
class UNet(nn.Module):
    def __init__(self, kernel, num_filters, num_colours=3, num_in_channels=1):
        super().__init__()

        # Useful parameters
        stride = 2
        padding = kernel // 2
        output_padding = 1

        self.downconv1 = nn.Sequential(
            nn.Conv2d(num_in_channels, num_filters, kernel_size=kernel, padding=padding),
            nn.BatchNorm2d(num_filters),
            nn.ReLU(),
            nn.Conv2d(num_filters, num_filters, kernel_size=kernel, padding=padding),
            nn.BatchNorm2d(num_filters),
            nn.ReLU(),
            nn.MaxPool2d(2),
        )
        self.downconv2 = nn.Sequential(
            nn.Conv2d(num_filters, num_filters*2, kernel_size=kernel, padding=padding),
            nn.BatchNorm2d(num_filters*2),
            nn.ReLU(),
            nn.Conv2d(num_filters*2, num_filters*2, kernel_size=kernel, padding=padding),
            nn.BatchNorm2d(num_filters*2),
            nn.ReLU(),
            nn.MaxPool2d(2),
        )

        self.rfconv = nn.Sequential(
            nn.Conv2d(num_filters*2, num_filters*2, kernel_size=kernel, padding=padding),
            nn.BatchNorm2d(num_filters*2),
            nn.ReLU(),
        )

        self.upconv1 = nn.Sequential(
            nn.ConvTranspose2d(num_filters*4, num_filters, kernel_size=kernel, stride=stride, padding=padding, output_padding=output_padding),
            nn.BatchNorm2d(num_filters),
            nn.ReLU(),
            nn.Conv2d(num_filters, num_filters, kernel_size=kernel, padding=padding),
            nn.BatchNorm2d(num_filters),
            nn.ReLU(),
        )
        self.upconv2 = nn.Sequential(
            nn.ConvTranspose2d(num_filters*2, num_colours, kernel_size=kernel, stride=stride, padding=padding, output_padding=output_padding),
            nn.BatchNorm2d(num_colours),
            nn.ReLU(),
        )
        self.finalconv = nn.Conv2d(num_colours, num_colours, kernel_size=kernel, padding=padding)

    def forward(self, x):
        x1 = self.downconv1(x)
        x2 = self.downconv2(x1)
        x3 = self.rfconv(x2)
        x4 = self.upconv1(torch.cat([x3, x2], 1))
        x5 = self.upconv2(torch.cat([x4, x1], 1))
        out = self.finalconv(x5)
        return out


### Part (ii) [2pt RESULTS] -  batch size of 100
Train the "UNet" model for the same amount of epochs as the previous CNN and plot the training curve using a batch size of 100.

In [ ]:
# Main training loop for UNet
args = AttrDict()
args_dict = {
    "gpu": True,
    "valid": False,
    "checkpoint": "",
    "colours": "./data/colours/colour_kmeans24_cat7.npy",
    "model": "UNet",
    "kernel": 3,
    "num_filters": 32,
    'learn_rate':0.001,
    "batch_size": 100,
    "epochs": 25,
    "seed": 0,
    "plot": True,
    "experiment_name": "colourization_cnn",
    "visualize": False,
    "downsize_input": False,
}
args.update(args_dict)
cnn = train(args)

**How does the result compare to the previous model? Did skip connections improve the validation loss? Did the skip connections improve the output qualitatively? How? Give at least two reasons why skip connections might improve the performance of our CNN models.**

**How does the result compare to the previous model? Did skip connections improve the validation loss?**
> The skip connections did improve the validation loss. 
> * The last validation loss after epoch 25 was 0.0044 (skip connection case) whereas the last validation loss at epoch 25 without skip connections was 0.0095.

**Did the skip connections improve the output qualitatively? How? Give at least two reasons why skip connections might improve the performance of our CNN models.**
* As the network becomes deeper, gradients calculated during backpropagation can become extremely small (vanish) due to multiple multiplications. This makes it hard for the weights to update and can stall the training process. **Skip connections allow the gradient to be directly backpropagated to earlier layers, preventing it from diminishing entirely.**
*  By allowing feature maps from earlier layers to be combined directly with those in deeper layers, skip connections help networks leverage both low-level features (like edges) and high-level features (like object parts) simultaneously. This can be particularly useful in tasks like image segmentation where fine details matter.
*  Without skip connections, training very deep networks can be challenging due to the vanishing and exploding gradient problems. Skip connections enable the construction and training of much deeper architectures, like the ResNet-152, which would be difficult to train otherwise.
*   In architectures like U-Net, skip connections allow features from downsampling layers to be fused with features from upsampling layers. This ensures that the spatial information lost during downsampling is reintroduced during upsampling, resulting in sharper and more accurate output.


### Part (iii) [2pt RESULTS] - different mini batch sizes
Re-train a few more "UNet" models using different mini batch sizes with a fixed number of epochs.

#### mini batch size of 50 - val loss 0.0043

In [ ]:
# Main training loop for UNet
args = AttrDict()
args_dict = {
    "gpu": True,
    "valid": False,
    "checkpoint": "",
    "colours": "./data/colours/colour_kmeans24_cat7.npy",
    "model": "UNet",
    "kernel": 3,
    "num_filters": 32,
    'learn_rate':0.001,
    "batch_size": 50,
    "epochs": 25,
    "seed": 0,
    "plot": True,
    "experiment_name": "colourization_cnn",
    "visualize": False,
    "downsize_input": False,
}
args.update(args_dict)
cnn = train(args)

Describe the effect of batch sizes on the training/validation loss, and the final image output.

In [ ]:
# Main training loop for UNet
args = AttrDict()
args_dict = {
    "gpu": True,
    "valid": False,
    "checkpoint": "",
    "colours": "./data/colours/colour_kmeans24_cat7.npy",
    "model": "UNet",
    "kernel": 3,
    "num_filters": 32,
    'learn_rate':0.001,
    "batch_size": 10,
    "epochs": 25,
    "seed": 0,
    "plot": True,
    "experiment_name": "colourization_cnn",
    "visualize": False,
    "downsize_input": False,
}
args.update(args_dict)
cnn = train(args)

**Describe the effect of batch sizes on the training/validation loss, and the final image output.**

> The lower the batch size, the lower the validation error we observed. However, it took significantly longer to train.
> Qualitatively speaking, the colors used by the colorization look more opaque than those of larger batch sizes.
> Smaller batch sizes can result in more diverse colorization results due to the increased noise in the gradient updates. This noise can act as a form of regularization, helping the model explore a wider range of colorization possibilities. The increased regularization from the noisy updates might help the model generalize better to new, unseen images, potentially leading to better colorization results on the validation and test sets.

# PART B - Conditional GAN

In this second half of the assignment we will construct a conditional generative adversarial network for our image colourization task. This second half of the assignment should be started after the lecture on generative adversarial networks (GANs).

## Part 1 Conditional GAN [10 Pt]

To start we will be modifying the previous sample code to construct and train a conditional GAN. Later you will have the opportunity to explore other modifications that can be made to our architecture to achieve the best results on image colourization model.


### Part (i) [2pt MODEL] - Implement Generator
Modify the provided training code to implement a generator. Then test to verify it works on the desired input

Hint: you can reuse your earlier autoencoder models here to act as a generator, if you do so, the architecture will be more like the pix2pix architecture (see https://phillipi.github.io/pix2pix/ for more details).

In [ ]:
# class Generator(nn.Module):
#     def __init__(self, kernel, num_filters, num_colours=3, num_in_channels=1):
#         super().__init__()

#         # Useful parameters
#         stride = 2
#         padding = kernel // 2
#         output_padding = 1

#         ############### YOUR CODE GOES HERE ###############
#         ###################################################




#     def forward(self, x):
#         ############### YOUR CODE GOES HERE ###############
#         ###################################################



#         return out

In [ ]:
class Generator(nn.Module):
    def __init__(self, kernel, num_filters, num_colours=3, num_in_channels=1):
        super().__init__()

        # Useful parameters
        stride = 2
        padding = kernel // 2
        output_padding = 1

        self.downconv1 = nn.Sequential(
            nn.Conv2d(num_in_channels, num_filters, kernel_size=kernel, padding=padding),
            nn.BatchNorm2d(num_filters),
            nn.ReLU(),
            nn.Conv2d(num_filters, num_filters, kernel_size=kernel, padding=padding),
            nn.BatchNorm2d(num_filters),
            nn.ReLU(),
            nn.MaxPool2d(2),
        )
        self.downconv2 = nn.Sequential(
            nn.Conv2d(num_filters, num_filters*2, kernel_size=kernel, padding=padding),
            nn.BatchNorm2d(num_filters*2),
            nn.ReLU(),
            nn.Conv2d(num_filters*2, num_filters*2, kernel_size=kernel, padding=padding),
            nn.BatchNorm2d(num_filters*2),
            nn.ReLU(),
            nn.MaxPool2d(2),
        )

        self.rfconv = nn.Sequential(
            nn.Conv2d(num_filters*2, num_filters*2, kernel_size=kernel, padding=padding),
            nn.BatchNorm2d(num_filters*2),
            nn.ReLU(),
        )

        self.upconv1 = nn.Sequential(
            nn.ConvTranspose2d(num_filters*4, num_filters, kernel_size=kernel, stride=stride, padding=padding, output_padding=output_padding),
            nn.BatchNorm2d(num_filters),
            nn.ReLU(),
            nn.Conv2d(num_filters, num_filters, kernel_size=kernel, padding=padding),
            nn.BatchNorm2d(num_filters),
            nn.ReLU(),
        )
        self.upconv2 = nn.Sequential(
            nn.ConvTranspose2d(num_filters*2, num_colours, kernel_size=kernel, stride=stride, padding=padding, output_padding=output_padding),
            nn.BatchNorm2d(num_colours),
            nn.ReLU(),
        )
        self.finalconv = nn.Conv2d(num_colours, num_colours, kernel_size=kernel, padding=padding)

    def forward(self, x):
        x1 = self.downconv1(x)
        x2 = self.downconv2(x1)
        x3 = self.rfconv(x2)
        # print(x3.shape, x2.shape)
        # print(x2.shape)
        x4 = self.upconv1(torch.cat([x3, x2], 1))
        x5 = self.upconv2(torch.cat([x4, x1], 1))
        out = self.finalconv(x5)
        return out


In [ ]:
#test generator architecture
G = Generator(3,32)
img_greyscale = torch.randn([50,1,32,32])
img_fake = G(img_greyscale)
print(img_fake.shape)

#### Training the Generator 

In [ ]:
import time
import os
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch.autograd import Variable

# Main training loop for GAN
args = AttrDict()
args_dict = {
    "gpu": True,
    "valid": False,
    "checkpoint": "",
    "colours": "./data/colours/colour_kmeans24_cat7.npy",
    "model": "Generator",
    "kernel": 3,
    "num_filters": 32,
    'learn_rate': 0.001,
    "batch_size": 100,
    "epochs": 25,
    "seed": 0,
    "plot": True,
    "experiment_name": "colourization_gan",
    "visualize": False,
    "downsize_input": False,
}

args.update(args_dict)
gen = train(args)


Write code to test the generator is working as intended.

In [ ]:
# TO BE COMPLETED






### Part (ii) [2pt MODEL] - Implement Discriminator
Modify the provided training code to implement a discriminator. Then test to verify it works on the desired input.

Hint: You should build a simple CNN model to act as a discriminator for real and fake images.

In [ ]:
# # discriminator code

# class Discriminator(nn.Module):
#     def __init__(self, kernel, num_filters, num_colours=3, num_in_channels=1):
#         super().__init__()

#         # Useful parameters
#         stride = 2
#         padding = kernel // 2
#         output_padding = 1

#         ############### YOUR CODE GOES HERE ###############
#         ###################################################


#     def forward(self, x, img_greyscale):

#         ############### YOUR CODE GOES HERE ###############
#         ###################################################



#         return out

In [ ]:
class Discriminator(nn.Module):
    def __init__(self, kernel, num_filters, num_colours=3, num_in_channels=1):
        super().__init__()

        # Useful parameters
        stride = 2
        padding = kernel // 2
        output_padding = 1

        ############### YOUR CODE GOES HERE ###############
        self.conv1 = nn.Conv2d(num_colours + 1, num_filters, kernel, stride, padding)
        self.conv2 = nn.Conv2d(num_filters, num_filters * 2, kernel_size=kernel, stride=stride, padding=padding)
        self.conv3 = nn.Conv2d(num_filters * 2, num_filters * 4, kernel_size=kernel, stride=stride, padding=padding)
        self.adaptive_pool = nn.AdaptiveAvgPool2d((8, 8))
        self.fc1 = nn.Linear(num_filters * 4 * 8 * 8, 1)

        self.leaky_relu = nn.LeakyReLU(0.2)
        self.sigmoid = nn.Sigmoid()
        ###################################################

    def forward(self, x, img_greyscale):
        ############### YOUR CODE GOES HERE ###############
        x = torch.cat((x, img_greyscale), 1)
        x = self.leaky_relu(self.conv1(x))
        x = self.leaky_relu(self.conv2(x))
        x = self.leaky_relu(self.conv3(x))
        x = self.adaptive_pool(x)
        x = x.view(x.size(0), -1)
        out = self.sigmoid(self.fc1(x))
        ###################################################

        return out


Write code to test that the discriminator is working as intended.

In [ ]:
# test discriminator architecture
img_colored = torch.randn([50,3,32,32])
img_greyscale = torch.randn([50,1,32,32])
D = Discriminator(3,32)
print(D(img_colored, img_greyscale).shape)

#### Test Discriminator

In [ ]:
def train_discriminator(args, disc=None, gen=None):
    # ... (previous code remains the same)
  
    # Numpy random seed
    np.random.seed(args.seed)
  
    # Save directory
    save_dir = "outputs/" + args.experiment_name
  
    # LOAD THE MODEL
    if disc is None:
        Disc = globals()[args.model]
        disc = Disc(args.kernel, args.num_filters)
  
    # LOSS FUNCTION
    criterion = nn.BCELoss()
    optimizer = torch.optim.Adam(disc.parameters(), lr=args.learn_rate)
  
    # DATA
    print("Loading data...")
    (x_train, y_train), (x_test, y_test) = load_cifar10()
  
    print("Transforming data...")
    train_rgb, train_grey = process(x_train, y_train, downsize_input=args.downsize_input)
    test_rgb, test_grey = process(x_test, y_test, downsize_input=args.downsize_input)
  
    # Create the outputs folder if not created already
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
  
    print("Beginning training ...")
    if args.gpu:
        if torch.backends.mps.is_available():
            mps_device = torch.device("mps")
            disc.to(mps_device)
        else:
            print("MPS device not available. Falling back to CPU.")
  
    start = time.time()
  
    train_losses = []
    valid_losses = []
    for epoch in range(args.epochs):
        # Train the Model
        disc.train()  # Change model to 'train' mode
        losses = []
        for i, (xs, ys) in enumerate(get_batch(train_grey, train_rgb, args.batch_size)):
            images, labels = get_torch_vars(xs, ys, args.gpu, mps_device if args.gpu else None)
            
            # Generate fake images
            fake_images = gen(images).detach()  # detach to avoid backpropagation to generator
            real_labels = Variable(torch.ones(images.size(0), 1)).to(images.device)
            fake_labels = Variable(torch.zeros(images.size(0), 1)).to(images.device)
            
            # Forward + Backward + Optimize for real images
            optimizer.zero_grad()
            outputs = disc(labels, images)
            real_loss = criterion(outputs, real_labels)
            real_loss.backward()
            
            # Forward + Backward + Optimize for fake images
            outputs = disc(fake_images, images)
            fake_loss = criterion(outputs, fake_labels)
            fake_loss.backward()
            
            optimizer.step()
            
            loss = real_loss + fake_loss
            losses.append(loss.data.item())
  
        train_losses.append(np.mean(losses))
  
        # Validate the Model
        disc.eval()  # Change model to 'eval' mode
        val_losses = []
        for i, (xs, ys) in enumerate(get_batch(test_grey, test_rgb, args.batch_size)):
            images, labels = get_torch_vars(xs, ys, args.gpu, mps_device if args.gpu else None)
            
            # Generate fake images
            fake_images = gen(images).detach()  # detach to avoid backpropagation to generator
            real_labels = Variable(torch.ones(images.size(0), 1)).to(images.device)
            fake_labels = Variable(torch.zeros(images.size(0), 1)).to(images.device)
            
            # Compute loss for real images
            outputs = disc(labels, images)
            real_loss = criterion(outputs, real_labels)
            
            # Compute loss for fake images
            outputs = disc(fake_images, images)
            fake_loss = criterion(outputs, fake_labels)
            
            loss = real_loss + fake_loss
            val_losses.append(loss.data.item())
  
        valid_losses.append(np.mean(val_losses))
  
        print(f'Epoch [{epoch+1}/{args.epochs}], '
              f'Training Loss: {train_losses[-1]:.4f}, '
              f'Validation Loss: {valid_losses[-1]:.4f}')
        
        # Visualization
        if args.visualize:
            # Sample a batch of images, labels, and generated images
            xs, ys = next(iter(get_batch(test_grey, test_rgb, 5)))
            images, labels = get_torch_vars(xs, ys, args.gpu, mps_device if args.gpu else None)
            fake_images = gen(images).detach()
            visual(images, labels, fake_images, gpu=args.gpu)
  
    # Plot the training and validation losses
    plt.figure(figsize=(10, 5))
    plt.plot(train_losses, label='Training Loss')
    plt.plot(valid_losses, label='Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Training and Validation Losses')
    plt.legend()
    plt.show()
  
    return disc

# Main training loop for GAN
args = AttrDict()
args_dict = {
    "gpu": True,
    "valid": False,
    "checkpoint": "",
    "colours": "./data/colours/colour_kmeans24_cat7.npy",
    "model": "Discriminator",
    "kernel": 3,
    "num_filters": 32,
    'learn_rate': 0.001,
    "batch_size": 100,
    "epochs": 25,
    "seed": 0,
    "plot": True,
    "experiment_name": "colourization_gan",
    "visualize": True,
    "downsize_input": False,
}

args.update(args_dict)
disc = train_discriminator(args, gen=gen)


### Part (iii) [2pt MODEL] - implement conditional GAN
Modify the provided training code to implement the training needed for a conditional GAN.

In [ ]:
def get_torch_vars(xs, ys, gpu=False):
    xs = torch.from_numpy(xs).float()
    ys = torch.from_numpy(ys).long()
    if gpu:
        device = torch.device("mps")
        xs = xs.to(device)
        ys = ys.to(device)
    return Variable(xs), Variable(ys)

In [ ]:
class AttrDict(dict):
    def __init__(self, *args, **kwargs):
        super(AttrDict, self).__init__(*args, **kwargs)
        self.__dict__ = self

def get_torch_vars(xs, ys, device=None):
    """
    Helper function to convert numpy arrays to pytorch tensors.
    If GPU or MPS is used, move the tensors to the specified device.

    Args:
      xs (float numpy tensor): greyscale input
      ys (int numpy tensor): categorical labels
      device (torch.device): the device to move the pytorch tensor to
    Returns:
      Variable(xs), Variable(ys)
    """
    xs = torch.from_numpy(xs).float()
    ys = torch.from_numpy(ys).float()
    if device is not None:
        xs = xs.to(device)
        ys = ys.to(device)
    return Variable(xs), Variable(ys)

def train(args, cnn=None):
    # Set the maximum number of threads to prevent crash in Teaching Labs
    # TODO: necessary?
    torch.set_num_threads(5)
    # Numpy random seed
    npr.seed(args.seed)

    # Save directory
    save_dir = "outputs/" + args.experiment_name

    # INPUT CHANNEL
    num_in_channels = 1 if not args.downsize_input else 3

    # LOAD THE MODEL
    if cnn is None:
        Net = globals()[args.model]
        cnn = Net(args.kernel, args.num_filters)
        discriminator = Discriminator(args.kernel, args.num_filters)

    # LOSS FUNCTION
    criterion = nn.BCELoss()
    g_optimizer = torch.optim.Adam(cnn.parameters(), lr=args.lr)
    d_optimizer = torch.optim.Adam(discriminator.parameters(), lr=args.lr)

    # DATA
    print("Loading data...")
    (x_train, y_train), (x_test, y_test) = load_cifar10()

    print("Transforming data...")
    train_rgb, train_grey = process(x_train, y_train, downsize_input=args.downsize_input)
    test_rgb, test_grey = process(x_test, y_test, downsize_input=args.downsize_input)

    # Create the outputs folder if not created already
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    device = torch.device("mps") if args.use_mps else torch.device("cuda" if args.gpu and torch.cuda.is_available() else "cpu")

    print("Beginning training ...")
    cnn.to(device)
    discriminator.to(device)

    start = time.time()

    train_losses = []
    valid_losses = []
    valid_accs = []
    for epoch in range(args.epochs):
        # Train the Model
        cnn.train()
        discriminator.train()
        losses = []

        for i, (xs, ys) in enumerate(get_batch(train_grey, train_rgb, args.batch_size)):
            images, labels = get_torch_vars(xs, ys, device)

            #--->ADDED 5
            img_grey = images
            img_real = labels
            batch_size = args.batch_size

            # discriminator training
            d_optimizer.zero_grad()

            D_real = discriminator(img_real, img_grey)
            labels = torch.zeros(batch_size).to(device)
            d_real_loss = criterion(D_real.squeeze(), labels)

            img_fake = cnn(img_grey)
            D_fake = discriminator(img_fake, img_grey)
            labels = torch.ones(batch_size).to(device)
            d_fake_loss = criterion(D_fake.squeeze(), labels)
            d_loss = d_real_loss + d_fake_loss

            d_loss.backward()
            d_optimizer.step()

            # generator training
            g_optimizer.zero_grad()

            img_fake = cnn(img_grey)
            D_fake = discriminator(img_fake, img_grey)
            labels = torch.zeros(batch_size).to(device)
            g_loss = criterion(D_fake.squeeze(), labels)

            g_loss.backward()
            g_optimizer.step()

        # print and visualize
        print(epoch, g_loss.cpu().detach(), d_loss.cpu().detach())
        visual(images, img_real, img_fake, args.gpu, 1)

    return cnn


In [ ]:
# Main training loop for GAN
args = AttrDict()
args_dict = {
    "gpu": True,
    "valid": False,
    "checkpoint": "",
    "colours": "./data/colours/colour_kmeans24_cat7.npy",
    "model": "Generator",  
    "kernel": 3,
    "num_filters": 32,
    'lr': 0.001,
    "batch_size": 100,
    "epochs": 25,
    "seed": 0,
    "plot": True,
    "experiment_name": "colourization_gan",
    "visualize": True,
    "downsize_input": False,
    "device": torch.device("mps"),
    "beta1": 0.5,
    "beta2": 0.999,
    "use_mps": True,  # Add this line
}


args.update(args_dict)
gan = train(args)


### Part (iv) [2pt RESULTS] - train cGAN and compare against autoencoder 
Train a conditional GAN for image colourization and comment on how the results compare to the autoencoder architectures from part 1.

> The colors that the network explores are definitely more diverse with the cGAN but the edges have become unclear. The results are also unstable. The colors change at a rather random pattern with different tones with little consistency.  

In [ ]:
args = AttrDict()
args_dict = {
    "gpu": True,
    "valid": False,
    "checkpoint": "",
    "colours": "./data/colours/colour_kmeans24_cat7.npy",
    "model": "Generator",
    "kernel": 3,
    "num_filters": 16,
    'lr':0.001,
    "batch_size": 50,
    "epochs": 100,
    "seed": 0,
    "plot": False,
    "experiment_name": "colourization_cnn",
    "visualize": False,
    "downsize_input": False,
    "use_mps": True, 
}
args.update(args_dict)
cnn = train(args)

#batch size of 50 with 100 epochs seamed to work

In [ ]:
# args = AttrDict()
# args_dict = {
#     "gpu": True,
#     "valid": False,
#     "checkpoint": "",
#     "colours": "./data/colours/colour_kmeans24_cat7.npy",
#     "gen_model": "Generator",
#     "disc_model": "Discriminator",
#     "kernel": 3,
#     "num_filters": 16,
#     'learn_rate': 0.001,
#     "batch_size": 50,
#     "epochs": 100,
#     "seed": 0,
#     "plot": False,
#     "experiment_name": "colourization_cgan",
#     "visualize": False,
#     "downsize_input": False,
# }
# args.update(args_dict)

# # Assuming `Generator` and `Discriminator` are the names of your generator and discriminator classes
# gen = Generator(args.num_filters, args.kernel)
# disc = Discriminator(args.num_filters, args.kernel)

# gen, disc = train(args, gen, disc)



### Part (v) [2pt DISCUSSION] - conditional GAN vs pix2pix

There are two ways to implement the conditional GAN. The first is a standard conditional GAN for which the generator is a decoder with an input that is random noise with a conditional. The second is the pix2pix architecture where the generator is a UNet architecture with a greyscale image input.

Briefly describe the pros and cons of the two different approaches in regards to the image colourization task.

Limit your answer to no more than 100 words.

#### Standard Conditional GAN

**Pros:**
* Simple architecture: The **generator is just a decoder that takes in random noise and a condition**, which makes the model easier to implement and train.
* Flexibility: Since the input is random noise, the generator can learn to generate a wide range of colorized images for a given greyscale image.

**Cons:**
* Lack of precision: The generated images may not be as precise or accurate as the ones generated by a pix2pix architecture since the input is random noise.
* If a Conditional GAN experiences mode collapse, it may generate similar colorized images for different grayscale input images, even if the ground truth colorized images are different. This would indicate that the generator is not effectively using the grayscale image as a condition to guide the colorization process.

#### Pix2Pix Architecture

**Pros:**
* Precision: The generator takes in a greyscale image as input, which allows it to generate more precise and accurate colorized images.
Better handling of complex scenes: The UNet architecture of the generator can handle complex scenes and fine details better than a standard conditional GAN.

**Cons:**
* Complexity: The architecture is more complex and resource intensive. 
* Potential for overfitting: Since the generator takes in a greyscale image as input, it may overfit and not generalize well. 

## Part 2 New Data [5 Pt]

### Part (i) [3pt RESULTS]

Retrieve sample pictures from online and demonstrate how well your best model performs. Provide all your code, including any image processing that you need to perform, and provide outputs on all the greyscale images you tested. Your outputs should show the greyscale images, ground truth, and model generated image.

In [ ]:
# TO BE COMPLETED





### Part (ii) [2pt DISCUSSION]

Summarize the qualitative performance of your best model on image colourization of new data and provide some reasons why it performs the way it does. Are there certain types of images on which your model works better than on others? Explain.



In [ ]:
'''
PROVIDE YOUR ANSWER BELOW






'''

## Part 3 Exploration [6 Pt]

At this point we have trained a few different generative models for our image colourization task with varying results. What makes this work exciting is that there many other approaches we could take. In this part of the assignment you are asked to consider some modifications you could make to improve the results and provide some comments on why those approaches could be beneficial.

### Part (i) [2pt DISCUSSION]

We've seen several times in this course how pretrained models could be used to improve performance on classification tasks. Do you think they could be beneficial in improving our results on the image colourization tasks? If so, would it be helpful to implement them in the discriminative and/or generative networks? Why or why not?

Limit your answers to no more than 100 words.

In [ ]:
'''
PROVIDE YOUR ANSWER BELOW






'''

### Part (ii) [2pt DISCUSSION]
The CIFAR10 images are 32x32 pixels in resolution. Do you believe that using similar images with a resolution of 250x250 would lead to better results. Why or why not?

Limit your answers to no more than 100 words.

In [ ]:
'''
PROVIDE YOUR ANSWER BELOW






'''


### Part (iii) [2pt DISCUSSION]

A colour space is a choice of mapping of colours into three-dimensional coordinates. Some colours could be close together in one colour space, but further apart in others. The RGB colour space is probably the most familiar to you, the model used in in our regression colourization example computes squared error in RGB colour space. But, most state of the art colourization models
do not use RGB colour space. How could using the RGB colour space be problematic? Your answer should relate how human perception of colour is different than the squared distance. You may use the Wikipedia article on colour space to help you answer the question.

Limit your answers to no more than 100 words.

In [ ]:
'''
PROVIDE YOUR ANSWER BELOW






'''

# PART C (Optional) - Bonus Challenge!

This is an optional exercise for those that finish the assignment early and would like to perform a deeper exploration of the concepts introduced in the assignment.

In Part A we constructed an autoencoder, specifically a UNet architecture with skip connections to learn to colourize images. In Part B completed the same task using conditional GANs. For this bonus challenge we will explore several other techniques that could be used to improve the model's performance on image colourization. A great tutorial on some of these different approaches can be found in a <a href="https://towardsdatascience.com/colorizing-black-white-images-with-u-net-and-conditional-gan-a-tutorial-81b2df111cd8">blog post by Moein Shariatnia</a>. It is highly recommended that you read the article before completing the following tasks:

Tasks:

1. Modify your conditional GAN architecture to incorporate a pretrained ResNet model to speed up and improve the generative abilities for image colourization.
2. Modify your conditional GAN architecture to incorporate the patch discriminator trained on local regions in the images.
3. Modify your conditional GAN architecture to use higher resolution images of upto 256 x 256 resolution and compare the performance. Summarize how performance changes with increasing resolution of images. Hint: for this task you will need to use another dataset, perhaps COCO, or ImageNet.
4. Modify your conditional GAN architecture to incorporate lab colour space image data representation instead of RGB data. Compare the performance of your model using RGB vs lab colour space data.

Bonus marks will be provided based on the number of tasks completed and how well they are completed. Summarize below your results and anything intersting you learned from the steps that you completed. Bonus marks cannot be accumulated beyond a maximum assignment grade.

In [ ]:
# TO BE COMPLETED







In [ ]:
'''
PROVIDE YOUR ANSWER BELOW






'''

### Saving to HTML
Detailed instructions for saving to HTML can be found <a href="https://stackoverflow.com/questions/53460051/convert-ipynb-notebook-to-html-in-google-colab/64487858#64487858">here</a>. Provided below are a summary of the instructions:

(1) download your ipynb file by clicking on File->Download.ipynb

(2) reupload your file to the temporary Google Colab storage (you can access the temporary storage from the tab to the left)

(3) run the following:

In [ ]:
%%shell
jupyter nbconvert --to html /content/A3.ipynb

(4) the html file will be available for download in the temporary Google Colab storage

(5) review the html file and make sure all the results are visible before submitting your assignment to Quercus

# Assignment Grading Rubric
The grading of the assignment will be based on the following categories:

(1) **10 Pt - EXPLORATORY QUESTIONS** These are basic questions that in most cases can be answered without requiring a fully working and trained neural network model. For example, data loading, processing and visualization, summary statistics, data exploration, model and training setup, etc.

(2) **10 Pt - MODEL** Student has successfully implemented all the required neural network models and has demonstrated successful training of the model without any errors.

(3) **10 Pt - RESULT** Students are evaluated based on the results achieved in comparison to the expected results of the assignment.

(4) **10 Pt - DISCUSSION QUESTIONS** Student demonstrated understanding beyond the basic exploratory questions, can answer some of the more challenging questions, and provide arguments for their model selection decisions.

(5) **10 Pt - COMMUNICATION** Student has provided a quality submission that is easy to read without too many unnecessary output statements that distract the reading of the document. The code has been well commented and all the answers are communicated clearly and concisely.

(6) **10 Pt - BONUS** Student has completed the assignment and has taken on the challenging bonus tasks listed in PART C. The student has demonstrated a good understanding of all aspects of the assignment and has exceeded expectations for the assignment.



**TOTAL GRADE = _____ of 50 Pts**